In [ ]:
import logging
from pyspark.sql import functions as F

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

try:
    # Load data from Unity Catalog tables
    hospital_stats_df = spark.table("catalog.db.hospital_stats_north_america")
    sales_associates_df = spark.table("catalog.db.sales_associates_employment_details")
    compensation_guidelines_df = spark.table("catalog.db.compensation_guidelines")
    hospital_sales_assignments_df = spark.table("catalog.db.hospital_sales_assignments")
    logistics_channels_df = spark.table("catalog.db.logistics_channels")
    growth_opportunities_df = spark.table("catalog.db.growth_opportunities")
    historical_sales_df = spark.table("catalog.db.historical_sales")
    company_goals_df = spark.table("catalog.db.company_goals")

    # Join Sales Associates and Compensation Guidelines
    compensation_df = sales_associates_df.join(
        compensation_guidelines_df, "Associate_ID", "inner"
    ).select(
        "Associate_ID", "Associate_Name", "Base_Salary", "Commission_Percentage", "Bonus"
    ).withColumn(
        "Compensation", F.expr("Base_Salary + (Commission_Percentage * Base_Salary) + Bonus")
    )

    # Integrate Hospital Stats with Sales Assignments
    hospital_sales_df = hospital_stats_df.join(
        hospital_sales_assignments_df, ["Hospital_ID", "Hospital_Name"], "inner"
    ).select(
        "Hospital_ID", "Hospital_Name", "Director_Name", "Manager_Name", "Associate_ID", "Associate_Name"
    )

    # Join Logistics Channels and Growth Opportunities
    growth_channels_df = logistics_channels_df.join(
        growth_opportunities_df, ["Channel_ID", "Channel_Type"], "inner"
    ).select(
        "Channel_ID", "Channel_Type", "Hospital_ID", "Growth_Opportunities", "Projected_Growth_Rate"
    )

    # Calculate Projected Revenue
    projected_revenue_df = historical_sales_df.withColumn(
        "Projected Revenue", F.expr("""
            CASE 
                WHEN Target_Year = 2024 THEN Sales_Revenue * (Projected_Sales_Growth_Rate / 100)
                WHEN Target_Year = 2025 THEN Sales_Revenue * (1 + Projected_Sales_Growth_Rate / 100)
                WHEN Target_Year = 2026 THEN Sales_Revenue * (1 + Projected_Sales_Growth_Rate / 100)
                ELSE Sales_Revenue
            END
        """)
    )

    # Generate Target Year Field
    target_year_df = projected_revenue_df.withColumn(
        "Target Year", F.expr("2023 + floor(rand() * 4)")
    )

    # Filter Target Year Greater than 2023
    filtered_df = target_year_df.filter(F.col("Target Year") > 2023)

    # Sort by Target Year
    sorted_df = filtered_df.orderBy("Target Year")

    # Write the final output to Unity Catalog tables
    sorted_df.write.format("delta").mode("overwrite").saveAsTable("catalog.db.target_sales_report")

    logger.info("ETL process completed successfully.")

except Exception as e:
    logger.error(f"Error during ETL process: {e}")
